In [21]:
1 + 1

2

In [1]:
from my_tools import *

## Trying to parallelize calculating balance deltas

In [117]:
def parseBlock(b):
    result = []
    for out in b.outputs:
        o = {'type': out.address.type, 'num':out.address.address_num, 'val': out.value }
        result.append(o)
    for in_ in b.inputs:
        i = {'type': in_.address.type, 'num':in_.address.address_num, 'val': -in_.value }
        result.append(i)
    return result

def parseBlocks(blocks):
    return [parseBlock(b) for b in blocks]

In [152]:
def parseBlock2(b):
    return {'o': b.outputs, 'i': b.inputs}

def parseBlocks2(blocks):
    return [parseBlock(b) for b in blocks]

In [157]:
def parseBlock3(b):
    return b.txes

def parseBlocks3(blocks):
    return [parseBlock3(b) for b in blocks]

In [158]:
%time parsed3_m = parseBlocks3(MNC.blocks)

CPU times: user 1.34 s, sys: 0 ns, total: 1.34 s
Wall time: 1.28 s


In [118]:
parseBlocks(MNC.blocks[:2])

[[{'num': 1, 'type': address_type.pubkey, 'val': 5000000000}],
 [{'num': 2, 'type': address_type.pubkey, 'val': 50000000000}]]

In [7]:
json.dumps(collections.defaultdict(lambda: 0.))

'{}'

In [8]:
import multiprocessing as mp
import joblib
from joblib import Parallel, delayed

def getClustersBalanceDeltasNp(blocks, map_address2cluster):
    balance_deltas = collections.defaultdict(lambda: 0.)
    for b in blocks:
        for out in b.outputs:
            add = out.address
            cluster_idx = map_address2cluster[add.type][add.address_num]
            balance_deltas[cluster_idx] += out.value
        for in_ in b.inputs:
            add = in_.address
            cluster_idx = map_address2cluster[add.type][add.address_num]
            balance_deltas[cluster_idx] -= in_.value
    return balance_deltas

def getClustersBalanceDeltasNpDebug(blocks, map_address2cluster):
    #balance_deltas = collections.defaultdict(lambda: 0.)
    #balance_deltas = collections.defaultdict(lambda: 0.)
    #for b in blocks:
    #    for out in b.outputs:
    #        add = out.address
    #        cluster_idx = map_address2cluster[add.type][add.address_num]
    #        balance_deltas[cluster_idx] += out.value
    #    for in_ in b.inputs:
    #        add = in_.address
    #        cluster_idx = map_address2cluster[add.type][add.address_num]
    #        balance_deltas[cluster_idx] -= in_.value
    #return balance_deltas
    print(len(blocks))
    for b in blocks:
        a = b
        #for out in b.outputs:
            #add = out.address
            #a = map_address2cluster[add.type][add.address_num]
            #b = out.value
    return dict(deltas_m[100])

def getClustersBalanceDeltasByGroupsNp(blocks, map_address2cluster, interval):
    print('Eto ya :)')
    blocks_by_groups = [blocks[i: i + interval] for i in range(0, len(blocks), interval)]
    balance_deltas = [getClustersBalanceDeltasNp(g, map_address2cluster) for g in blocks_by_groups]
    return balance_deltas

def getClustersBalanceDeltasByGroupsNpPar(blocks, map_address2cluster, interval):
    blocks_by_groups = [list(blocks[i: i + interval]) for i in range(0, len(blocks), interval)]
    #print(blocks_by_groups)
    #pool = mp.Pool(processes=4)
    
    balance_deltas = Parallel(n_jobs=4, verbose=100)(delayed(getClustersBalanceDeltasNpDebug)(g, map_address2cluster) for g in blocks_by_groups)
    #balance_deltas = [pool.apply(getClustersBalanceDeltasNp, (g, map_address2cluster))
    #                  for g in blocks_by_groups]
    return balance_deltas

def getClustersBalanceDeltasByGroupsNpParAsync(blocks, map_address2cluster, interval):
    blocks_by_groups = [list(blocks[i: i + interval]) for i in range(0, len(blocks), interval)]
    print(len(blocks_by_groups))
    print(blocks_by_groups)
    pool = mp.Pool(processes=8)
    results = [pool.apply_async(getClustersBalanceDeltasNp, (g, map_address2cluster))
               for g in blocks_by_groups]
    balance_deltas = [p.get() for p in results]
    return balance_deltas

In [161]:
import multiprocessing as mp
import joblib
from joblib import Parallel, delayed

def getClustersBalanceDeltasNp(blocks, map_address2cluster):
    balance_deltas = collections.defaultdict(lambda: 0.)
    for b in blocks:
        for out in b.outputs:
            add = out.address
            cluster_idx = map_address2cluster[add.type][add.address_num]
            balance_deltas[cluster_idx] += out.value
        for in_ in b.inputs:
            add = in_.address
            cluster_idx = map_address2cluster[add.type][add.address_num]
            balance_deltas[cluster_idx] -= in_.value
    return dict(balance_deltas)

def getClustersBalanceDeltasNpDebug__(blocks, map_address2cluster):
    #balance_deltas = collections.defaultdict(lambda: 0.)
    #balance_deltas = collections.defaultdict(lambda: 0.)
    #for b in blocks:
    #    for out in b.outputs:
    #        add = out.address
    #        cluster_idx = map_address2cluster[add.type][add.address_num]
    #        balance_deltas[cluster_idx] += out.value
    #    for in_ in b.inputs:
    #        add = in_.address
    #        cluster_idx = map_address2cluster[add.type][add.address_num]
    #        balance_deltas[cluster_idx] -= in_.value
    #return balance_deltas
    #print(len(blocks))
    balance_deltas = collections.defaultdict(lambda: 0.)
    for b in blocks:
        for io in b:
            cluster_idx = map_address2cluster[io['type']][io['num']]
            balance_deltas[cluster_idx] += io['val']
    return dict(deltas_m[100])

def getClustersBalanceDeltasByGroupsNpPar__(blocks, map_address2cluster, interval):
    blocks_by_groups = [parseBlocks(blocks[i: i + interval]) for i in range(0, len(blocks), interval)]
    #print(blocks_by_groups)
    #pool = mp.Pool(processes=4)
    
    balance_deltas = Parallel(n_jobs=4, verbose=100)(delayed(getClustersBalanceDeltasNpDebug__)(g, map_address2cluster) for g in blocks_by_groups)
    #balance_deltas = [pool.apply(getClustersBalanceDeltasNp, (g, map_address2cluster))
    #                  for g in blocks_by_groups]
    return balance_deltas


def getClustersBalanceDeltasNpDebug__2(blocks, map_address2cluster):
    #balance_deltas = collections.defaultdict(lambda: 0.)
    #balance_deltas = collections.defaultdict(lambda: 0.)
    #for b in blocks:
    #    for out in b.outputs:
    #        add = out.address
    #        cluster_idx = map_address2cluster[add.type][add.address_num]
    #        balance_deltas[cluster_idx] += out.value
    #    for in_ in b.inputs:
    #        add = in_.address
    #        cluster_idx = map_address2cluster[add.type][add.address_num]
    #        balance_deltas[cluster_idx] -= in_.value
    #return balance_deltas
    #print(len(blocks))
    balance_deltas = collections.defaultdict(lambda: 0.)
    for b in blocks:
        for out in b['o']:
            add = out.address
            cluster_idx = map_address2cluster[add.type][add.address_num]
            balance_deltas[cluster_idx] += out.value
        for in_ in b['i']:
            add = in_.address
            cluster_idx = map_address2cluster[add.type][add.address_num]
            balance_deltas[cluster_idx] -= in_.value
    return dict(deltas_m[100])

def getClustersBalanceDeltasByGroupsNpPar__2(blocks, map_address2cluster, interval):
    blocks_by_groups = [parseBlocks2(blocks[i: i + interval]) for i in range(0, len(blocks), interval)]
    #print(blocks_by_groups)
    #pool = mp.Pool(processes=4)
    
    balance_deltas = Parallel(n_jobs=4, verbose=100)(delayed(getClustersBalanceDeltasNpDebug__2)(g, map_address2cluster) for g in blocks_by_groups)
    #balance_deltas = [pool.apply(getClustersBalanceDeltasNp, (g, map_address2cluster))
    #                  for g in blocks_by_groups]
    return balance_deltas

def getClustersBalanceDeltasNpDebug__3(blocks, map_address2cluster):
    #balance_deltas = collections.defaultdict(lambda: 0.)
    balance_deltas = collections.defaultdict(lambda: 0.)
    for b in blocks:
        for out in b.outputs:
            add = out.address
            cluster_idx = map_address2cluster[add.type][add.address_num]
            balance_deltas[cluster_idx] += out.value
        for in_ in b.inputs:
            add = in_.address
            cluster_idx = map_address2cluster[add.type][add.address_num]
            balance_deltas[cluster_idx] -= in_.value
    return balance_deltas

def getClustersBalanceDeltasByGroupsNpPar__3(blocks, map_address2cluster, interval):
    blocks_by_groups = [parseBlocks3(blocks[i: i + interval]) for i in range(0, len(blocks), interval)]
    #print(blocks_by_groups)
    #pool = mp.Pool(processes=4)
    
    balance_deltas = Parallel(n_jobs=4, verbose=100)(delayed(getClustersBalanceDeltasNpDebug__3)(g, map_address2cluster) for g in blocks_by_groups)
    #balance_deltas = [pool.apply(getClustersBalanceDeltasNp, (g, map_address2cluster))
    #                  for g in blocks_by_groups]
    return balance_deltas

In [3]:
def sumClustersBalanceDeltas(clustersBalanceDeltas, clusters_count):
    balance_sums = np.zeros(clusters_count)
    for deltas in clustersBalanceDeltas:
        for cl_idx in deltas:
            balance_sums += deltas[cl_idx]
    return balance_sums

In [4]:
# Q_MINCOIN
MNC = CoinDataMgr(blocksci_path="/home/ubuntu/mincoin",
                  path_to_clusters="/home/ubuntu/~/clusters2",
                  folder_with_calculated="/home/ubuntu/thesis_work/calculated/mincoin_first_500k_blocks",
                  max_block=500000)

Got 500000 blocks
Got 90517 clusters


In [5]:
MNC[S_MAP_A2C].readAll()

Reading folder '/home/ubuntu/thesis_work/calculated/mincoin_first_500k_blocks' with prefix 'int1000b_map_address2cluster'...
  Got 'int1000b_map_address2cluster.hdf5', key is ''
Finish


In [162]:
%time deltas_par_m = getClustersBalanceDeltasByGroupsNpPar__2(MNC.blocks[:1], MNC[S_MAP_A2C][''], MNC.group_size)

Pickling array (shape=(100000,), dtype=int32).
Memmaping (shape=(600000,), dtype=int32) to new file /dev/shm/joblib_memmaping_pool_15936_140303565495096/15936-140303347763856-266b86af8f90c58c4d333c72aabccdbf.pkl
Memmaping (shape=(600000,), dtype=int32) to old file /dev/shm/joblib_memmaping_pool_15936_140303565495096/15936-140303347763856-266b86af8f90c58c4d333c72aabccdbf.pkl
Memmaping (shape=(600000,), dtype=int32) to old file /dev/shm/joblib_memmaping_pool_15936_140303565495096/15936-140303347763856-266b86af8f90c58c4d333c72aabccdbf.pkl
Pickling array (shape=(100000,), dtype=int32).
Pickling array (shape=(100000,), dtype=int32).
Pickling array (shape=(100000,), dtype=int32).
Memmaping (shape=(600000,), dtype=int32) to old file /dev/shm/joblib_memmaping_pool_15936_140303565495096/15936-140303347763856-266b86af8f90c58c4d333c72aabccdbf.pkl
Pickling array (shape=(100000,), dtype=int32).


JoblibTypeError: JoblibTypeError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/lib/python3.5/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    179         sys.exit(msg)
    180     main_globals = sys.modules["__main__"].__dict__
    181     if alter_argv:
    182         sys.argv[0] = mod_spec.origin
    183     return _run_code(code, main_globals, None,
--> 184                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.5/dist-packages/ipykernel_launcher.py')
    185 
    186 def run_module(mod_name, init_globals=None,
    187                run_name=None, alter_sys=False):
    188     """Execute a module's code without importing it

...........................................................................
/usr/lib/python3.5/runpy.py in _run_code(code=<co

In [137]:
%time deltas_par_m = getClustersBalanceDeltasByGroupsNpPar(MNC.blocks[:1000], MNC[S_MAP_A2C][''], MNC.group_size)

1000
Pickling array (shape=(100000,), dtype=int32).
Memmaping (shape=(600000,), dtype=int32) to new file /dev/shm/joblib_memmaping_pool_15936_140304774188280/15936-140303388715608-266b86af8f90c58c4d333c72aabccdbf.pkl
Memmaping (shape=(600000,), dtype=int32) to old file /dev/shm/joblib_memmaping_pool_15936_140304774188280/15936-140303388715608-266b86af8f90c58c4d333c72aabccdbf.pkl
Memmaping (shape=(600000,), dtype=int32) to old file /dev/shm/joblib_memmaping_pool_15936_140304774188280/15936-140303388715608-266b86af8f90c58c4d333c72aabccdbf.pkl
Pickling array (shape=(100000,), dtype=int32).
Pickling array (shape=(100000,), dtype=int32).
Pickling array (shape=(100000,), dtype=int32).
Memmaping (shape=(600000,), dtype=int32) to old file /dev/shm/joblib_memmaping_pool_15936_140304774188280/15936-140303388715608-266b86af8f90c58c4d333c72aabccdbf.pkl
Pickling array (shape=(100000,), dtype=int32).
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Batch computation

In [30]:
%time deltas_m = getClustersBalanceDeltasByGroupsNp(MNC.blocks, MNC[S_MAP_A2C][''], MNC.group_size)

CPU times: user 17.7 s, sys: 0 ns, total: 17.7 s
Wall time: 17.3 s


In [86]:
%time deltas2_m = getClustersBalanceDeltasByGroupsNp(MNC.blocks, MNC[S_MAP_A2C][''], MNC.group_size)

CPU times: user 18.1 s, sys: 0 ns, total: 18.1 s
Wall time: 17.4 s


In [87]:
deltas2_m[0]

{2339: 5000000000.0,
 2340: 2500000000000.0,
 2341: 11500000000000.0,
 2342: 1000000000000.0,
 2343: 150000000000.0,
 2344: 250000000000.0,
 2345: 2500000000000.0,
 2346: 100000000000.0,
 2347: 50000000000.0,
 2348: 50000000000.0,
 2349: 4250000000000.0,
 2350: 1500000000000.0,
 2351: 400000000000.0,
 2352: 50000000000.0,
 2353: 100000000000.0,
 2354: 300000000000.0,
 2355: 50000000000.0,
 2356: 9450000000000.0,
 2357: 15400000000000.0,
 2358: 300000000000.0,
 2359: 50000000000.0}

In [166]:
LTC[S_MAP_A2C].read('np')

There is already np version of data, stop


In [63]:
deltas_m[4]

defaultdict(<function __main__.getClustersBalanceDeltasNp.<locals>.<lambda>>,
            {2341: 25131890000000.0,
             2345: -2500000000000.0,
             2356: 185600000000.0,
             2357: -24997980000000.0,
             2358: 23010000000.0,
             2359: 5000000000.0,
             2362: 20000000000.0,
             2364: 25000000000.0,
             2397: 10000000000.0,
             2430: 15000000000.0,
             2432: 5000000000.0,
             2441: 5000000000.0,
             2443: 52000000000.0,
             2445: 5000000000.0,
             2471: 90000000000.0,
             2478: 122000000000.0,
             2486: 367270000000.0,
             2498: 800000000.0,
             2532: 311000000000.0,
             2546: 283800000000.0,
             2547: 5000000000.0,
             2548: 5000000000.0,
             2549: 5000000000.0,
             2550: 5000000000.0,
             2551: 5000000000.0,
             2552: 5000000000.0,
             2553: 200000000.0,
   

In [81]:
d = dict(collections.defaultdict(lambda: 0.))

In [66]:
d['a'] += 5

In [82]:
d

{}

In [79]:
def A2():
    print(collections.defaultdict(lambda: 0.))
    balance_deltas = collections.defaultdict(lambda: 0.)
    print(balance_deltas)
    return balance_deltas
A2()

defaultdict(<function A2.<locals>.<lambda> at 0x7f9c0411cd90>, {})
defaultdict(<function A2.<locals>.<lambda> at 0x7f9c0411cd90>, {})


defaultdict(<function __main__.A2.<locals>.<lambda>>, {})

In [80]:
print(collections.defaultdict(lambda: 0.))

defaultdict(<function <lambda> at 0x7f9c0411ce18>, {})


In [44]:
%time deltas_m = getClustersBalanceDeltasByGroupsNpParAsync(MNC.blocks[:1], MNC[S_MAP_A2C][''], MNC.group_size)

1
[[Block(numTxes=1, height=0, header_hash=12a765e31ffd4059bada1e25190f6e98c99d9714d334efa41a195a7e7e04bfe2, version=1, timestamp=1317972665, bits=0, nonce=2084524493)]]


MaybeEncodingError: Error sending result: 'defaultdict(<function getClustersBalanceDeltasNp.<locals>.<lambda> at 0x7f9c4c664e18>, {})'. Reason: 'AttributeError("Can't pickle local object 'getClustersBalanceDeltasNp.<locals>.<lambda>'",)'

Process ForkPoolWorker-811:
Process ForkPoolWorker-810:
Process ForkPoolWorker-809:
Process ForkPoolWorker-812:
Process ForkPoolWorker-806:
Process ForkPoolWorker-808:
Process ForkPoolWorker-807:
Traceback (most recent call last):
Process ForkPoolWorker-805:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.5/multip

## Попытка проверить, что балансы начали "правильно" считаться после учёта комиссии

In [ ]:
def getClustersBalanceDeltasNp(blocks, map_address2cluster):
    balance_deltas = collections.defaultdict(lambda: 0)
    for b in blocks:
        for out in b.outputs:
            add = out.address
            cluster_idx = map_address2cluster[add.type][add.address_num]
            balance_deltas[cluster_idx] += out.value
        for in_ in b.inputs:
            add = in_.address
            cluster_idx = map_address2cluster[add.type][add.address_num]
            balance_deltas[cluster_idx] -= in_.value
    return balance_deltas

def getClustersBalanceDeltasByGroupsNp(blocks, map_address2cluster, interval):
    print('Eto ya :)')
    blocks_by_groups = [blocks[i: i + interval] for i in range(0, len(blocks), interval)]
    balance_deltas = [getClustersBalanceDeltasNp(g, map_address2cluster) for g in blocks_by_groups]
    return balance_deltas

def getClustersBalanceDeltasNpCorrect(blocks, map_address2cluster):
    balance_deltas = collections.defaultdict(lambda: 0)
    for b in blocks:
        for out in b.outputs:
            if out.tx.is_coinbase:
                continue
            add = out.address
            cluster_idx = map_address2cluster[add.type][add.address_num]
            balance_deltas[cluster_idx] += out.value
        for in_ in b.inputs:
            #if in_.tx.is_coinbase:
            #    continue
            add = in_.address
            cluster_idx = map_address2cluster[add.type][add.address_num]
            balance_deltas[cluster_idx] -= in_.value
    return balance_deltas

def getClustersBalanceDeltasByGroupsNpCorrect(blocks, map_address2cluster, interval):
    print('Eto ya :)')
    blocks_by_groups = [blocks[i: i + interval] for i in range(0, len(blocks), interval)]
    balance_deltas = [getClustersBalanceDeltasNpCorrect(g, map_address2cluster) for g in blocks_by_groups]
    return balance_deltas

def getClustersBalanceDeltasNpCorrect2(blocks, map_address2cluster):
    balance_deltas = collections.defaultdict(lambda: 0)
    for b in blocks:
        if b.fee > 0:
            miner = guessMinerAdds3((b,))[0]
            if miner != UNKNOWN:
                cluster_idx = map_address2cluster[miner.type][miner.address_num]
                balance_deltas[cluster_idx] += b.fee
        for out in b.outputs:
            if out.tx.is_coinbase:
                continue
            add = out.address
            cluster_idx = map_address2cluster[add.type][add.address_num]
            balance_deltas[cluster_idx] += out.value
        for in_ in b.inputs:
            #if in_.tx.is_coinbase:
            #    continue
            add = in_.address
            cluster_idx = map_address2cluster[add.type][add.address_num]
            balance_deltas[cluster_idx] -= in_.value
    return balance_deltas

def getClustersBalanceDeltasByGroupsNpCorrect2(blocks, map_address2cluster, interval):
    print('Eto ya :)')
    blocks_by_groups = [blocks[i: i + interval] for i in range(0, len(blocks), interval)]
    balance_deltas = [getClustersBalanceDeltasNpCorrect2(g, map_address2cluster) for g in blocks_by_groups]
    return balance_deltas

In [ ]:
def getFlowAndIncomeVolumeBase(blocks, map_address2cluster):
    balance_deltas = collections.defaultdict(lambda: 0)

    for b in blocks:
        for out in b.outputs:
            if out.address.type == blocksci.address_type.nulldata:
                continue
            if out.tx.is_coinbase:
                cluster_idx = 'income'
            else:
                add = out.address
                cluster_idx = map_address2cluster[add.type][add.address_num]
            balance_deltas[cluster_idx] += out.value
        for in_ in b.inputs:
            #if in_.tx.is_coinbase:
            #    continue
            add = in_.address
            cluster_idx = map_address2cluster[add.type][add.address_num]
            balance_deltas[cluster_idx] -= in_.value
    
    income = balance_deltas['income']
    flow = sum([abs(delta) for delta in balance_deltas.values()]) - income
    return flow, income

def getFlowAndIncomeVolume(blocks, map_address2cluster, interval):
    flows, incomes = [], []
    for i in range(0, len(blocks), interval):
        cur_blocks = blocks[i: i + interval]
        f, i = getFlowAndIncomeVolumeBase(cur_blocks, map_address2cluster)
        flows.append(f)
        incomes.append(i)
    return flows, incomes

def getRelativeFlowVolumes_dedicated(flows, incomes):
    relativeFlowVolumes = []
    total_income = 0
    for i in len(flows):
        total_income += incomes[i]
        relativeFlowVolumes.append(flows[i] / total_income)
    return relativeFlowVolumes

In [ ]:
%time flows_par_m, incomes_par_m = getFlowAndIncomeVolumePar(MNC.chain, MNC[S_MAP_A2C][''], MNC.group_size, end=500000)

In [ ]:
def sumClustersBalanceDeltas(clustersBalanceDeltas, clusters_count):
    balance_sums = np.zeros(clusters_count)
    for deltas in clustersBalanceDeltas:
        for cl_idx in deltas:
            balance_sums += deltas[cl_idx]
    return balance_sums

In [ ]:

def checkFlowSum2(blocks):
    flowSum = 0
    for b in blocks:
        for out in b.outputs:
            if out.tx.is_coinbase:
                continue
            flowSum += out.value
        for in_ in b.inputs:
            if in_.tx.is_coinbase:
                continue
            flowSum -= in_.value
        flowSum += b.fee
    return flowSum

In [ ]:
def getRelativeFlowVolumes(balance_deltas, map_address2cluster):
    relativeFlowVolumes = []
    overall_current_alive_coins = 0
    
    nulldata_map = map_address2cluster[blocksci.address_type.nulldata]
    nulldata_cluster_ids = set(nulldata_map[1:nulldata_map[0]])
    
    for group_deltas in balance_deltas:
        emitted_volume = sum(group_deltas.values())
        normal_ids_balances = {group_deltas[cl_id] for cl_id in group_deltas
                               if not cl_id in nulldata_cluster_ids}
        summary_balances_change = sum([abs(bal) for bal in normal_ids_balances])
        overall_current_alive_coins += sum(normal_ids_balances)
        
        relativeFlowVolumes.append((summary_balances_change - emitted_volume) / overall_current_alive_coins)
    return relativeFlowVolumes

In [ ]:
%time deltas_correct_m = getClustersBalanceDeltasByGroupsNpCorrect(MNC.blocks, MNC[S_MAP_A2C][''], MNC.group_size)

In [ ]:
b = MNC.blocks[3804]
f = 0
for out in b.outputs:
    #if out.tx.is_coinbase:
    #    continue
    f += out.value
    #print(out.value, out.address)
for in_ in b.inputs:
    if in_.tx.is_coinbase:
        continue
    f -= in_.value
    #print(-in_.value, in_.address)
#f

In [ ]:
bad_tx = MNC.chain.tx_with_index(3811)
bad_tx.output_value - bad_tx.input_value

In [ ]:
# SUKA 2.5h
%time deltas_b = getClustersBalanceDeltasByGroupsNp(BTC.blocks, BTC[S_MAP_A2C][''], BTC.group_size)


with open('allah_deltas_b' + '.json', 'w') as fout:
    fout.write
    for dd in deltas_b:
        str_dict = {str(k):v for k,v in dd.items()}
        fout.write(json.dumps(str_dict))
        fout.write('\n')

        
ind = 0
with open('allah_deltas_b' + '.json', 'r') as fin:
    for line in fin:
        str_dict = json.loads(line)
        if str_dict != {str(k):v for k,v in deltas_b[ind].items()}:
            print('Fuck!')
            break
        ind += 1
        #test_dict.append(str_dict)

In [ ]:
import pandas as pd
prices_l_df = pd.read_csv('~/thesis_work/downloaded/ltc-usd-max.csv')
prices_half_l_df = prices_l_df[pd.to_datetime(prices_l_df['snapped_at']) <= LTC.times[-1]]
prices_half_l_df[:3]

In [ ]:
prices_l_df_to_save = pd.DataFrame({'times': prices_half_l_df['snapped_at'],
                                    'prices': prices_half_l_df['price']})
prices_l_df_to_save[:1]